# Extract

In [91]:
import pandas as pd
active_corporations=pd.read_csv(r'Active_Corporations___Beginning_1800.csv')
unemployment=pd.read_csv(r'Local_Area_Unemployment_Statistics__Beginning_1976.csv')
counties=pd.read_csv(r'counties.csv')

In [92]:
counties.head()

,County,FIPS Code[1],County Seat[2],Created[2],Formed from[3],Named for[4],Density,2010 Population[2],Area[2],Map
0,Albany County,1,Albany,1683,One of 12 original counties created in the New...,James II of England (James VII of Scotland) (1...,570.74,"304,204","533 sq mi(1,380 km2)",NaN
1,Allegany County,3,Belmont,1806,Genesee County,A variant spelling of the Allegheny River,47.34,"48,946","1,034 sq mi(2,678 km2)",NaN
2,Bronx County,5,New York City (coextensive withThe Bronx),1914[5],New York County,"Jonas Bronck (1600?–1643), an early settler of...","24,118.2","1,385,108",57.43 sq mi(149 km2),NaN
3,Broome County,7,Binghamton,1806,Tioga County,"John Broome (1738–1810), fourth Lieutenant Gov...",280.56,"200,600","715 sq mi(1,852 km2)",NaN
4,Cattaraugus County,9,Little Valley,1808,Genesee County,"A Seneca word meaning ""bad smelling banks"", re...",61.31,"80,317","1,310 sq mi(3,393 km2)",NaN


# Transform

## Cleaning corporations dataset


In [95]:
not_null_date=active_corporations['Initial DOS Filing Date'].isnull().map(lambda x: not x)
active_corporations=active_corporations[not_null_date]
active_corporations['month']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[0]))
active_corporations['day']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[1]))
active_corporations['year']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[2]))
active_corporations=active_corporations[active_corporations['year'].map(lambda x: x>1976)]

In [96]:
def format_county(s):
    suffix=' County'
    if s.endswith(suffix):
        return s[:-len(suffix)].lower()
    return s.lower()
county_names=counties['County']
county_names=set(map(format_county,county_names))

In [97]:
not_null_county=active_corporations['County'].isnull().map(lambda x: not x)
active_corporations=active_corporations[not_null_county]
active_corporations['County']=active_corporations['County'].map(lambda x: format_county(x))
active_corporations=active_corporations[active_corporations['County'].map(lambda x: x in county_names)]

## Making dictionary to convert area to county

In [98]:
towns=pd.read_csv(r'towns.csv')
pairs=towns[['Town','County']]
pair_d=pairs.to_dict()
town_d={pair_d['Town'][i].lower():pair_d['County'][i].lower() for i in pair_d['Town'].keys()}

In [99]:
cities=pd.read_csv(r'cities.csv')
city_pairs=cities[['City','County']]
city_pair_d=city_pairs.to_dict()
city_d={city_pair_d['City'][i].lower():city_pair_d['County'][i].lower() for i in city_pair_d['City'].keys()}
town_d.update(city_d)

In [100]:
areas=unemployment['Area'].map(lambda x:format_county(x)).unique()
area2county={}
for area in areas:
    found=False
    for key in town_d.keys():
        if key in area:
            area2county[area]=town_d[key]
            found=True
            break
    for county in county_names:
        if found:
            break
        if area in county:
            area2county[area]=county
            found=True
            break
    if not found:
        print('area:',area)
print('found %s/%s' % (str(len(area2county)),str(len(areas))))

area: balance of state
area: capital region
area: de witt town
area: finger lakes
area: freeport village
area: lindenhurst
area: long island
area: north country
area: rockville centre
area: southern tier
area: spring valley village
area: valley stream
found 192/204


## Cleaning unemployment dataset

In [101]:
unemployment=unemployment[unemployment['Area'].map(lambda x: format_county(x) in area2county)]
unemployment['County']=unemployment['Area'].map(lambda x: area2county[format_county(x)])

In [102]:
unemployment.head()

,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate,County
0,Albany City,2021,2,46700,43000,3700,7.9,albany
1,Albany City,2021,1,46800,43100,3700,7.9,albany
2,Albany City,2020,12,47200,43500,3700,7.8,albany
3,Albany City,2020,11,47400,43800,3600,7.7,albany
4,Albany City,2020,10,47200,43500,3800,8.0,albany


## Merging Datasets

In [103]:
active_corporations.head()

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip,month,day,year
0,3211809,00:02:59 LLC,05/31/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,00:02:59 LLC,"656 UNION STREET, APT. 2",NaN,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005
1,4472700,000 LLC,10/15/2013,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,000,304 MESEROLE ST #2A,NaN,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,15,2013
2,4723823,"000 MEADOW LANE, LLC",03/11/2015,suffolk,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O ACKERMAN O'BRIEN PACHMAN & BROWN LLP,34 PANTIGO ROAD,NaN,EAST HAMPTON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,11,2015
3,5224240,00168 HOME CORP.,10/26/2017,suffolk,NEW YORK,DOMESTIC BUSINESS CORPORATION,00168 HOME CORP.,28 PHEASANT RUN LANE,NaN,DIX HILLS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,26,2017
4,5581065,"0016 HEMPSTEAD TURNPIKE, LLC",07/02/2019,suffolk,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"GOLDENBOCK, EISEMAN, ASSOR, BELL & PESKOE LLP",ATTN LARRY A WELCH ESQ,711 3RD AVE FL 16,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2,2019


In [112]:
joined_df=active_corporations.merge(unemployment,how='inner',left_on=['County','month','year'],right_on=['County','Month','Year'])

In [113]:
joined_df.head()

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,month,day,year,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate
0,3211809,00:02:59 LLC,05/31/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,00:02:59 LLC,"656 UNION STREET, APT. 2",NaN,BROOKLYN,...,5,31,2005,Kings County,2005,5,1061300,999600,61700,5.8
1,3203583,1010 DEVELOPMENT LLC,05/11/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O CHARALABOS P BAKALIS,8721 5TH AVE,SUITE 100,BROOKLYN,...,5,11,2005,Kings County,2005,5,1061300,999600,61700,5.8
2,3199867,103 MESEROLE LLC,05/03/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,103 MESEROLE LLC,107 CLYMER STREET,NaN,BROOKLYN,...,5,3,2005,Kings County,2005,5,1061300,999600,61700,5.8
3,3211699,"1059 E. 35TH STREET, LLC",05/31/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"1059 E. 35TH STREET, LLC",PO BOX 150-260,NaN,BROOKLYN,...,5,31,2005,Kings County,2005,5,1061300,999600,61700,5.8
4,3212110,"105 KINGS HIGHWAY CONDOMINIUM ASSOCIATION, INC.",05/31/2005,kings,NEW YORK,DOMESTIC NOT-FOR-PROFIT CORPORATION,"105 KINGS HIGHWAY CONDOMINIUM ASSOCIATION, INC.",SUITE 711,45 JOHN STREET,NEW YORK,...,5,31,2005,Kings County,2005,5,1061300,999600,61700,5.8


In [114]:
joined_df.shape

(10029155, 40)

In [115]:
active_corporations.shape

(2867449, 33)

# Load

In [116]:
joined_df.to_csv('joined_dataset.csv')